<a href="https://colab.research.google.com/github/s-zeidi/Natural-language-inference-NLI-/blob/main/Fine_Tune_Qwen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Clone LLaMA-Factory (download repo)
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory

Cloning into 'LLaMA-Factory'...
^C
[Errno 2] No such file or directory: 'LLaMA-Factory'
/content


In [ ]:
# STEP 2: Install Requirements
!pip install -r requirements.txt
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -e ".[torch,metrics]"
!pip install liger-kernel

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00
  Attempting uninstall: 

In [ ]:
# STEP 3: Mount Google Drive (you will be asked to authorize)
from google.colab import drive
drive.mount('/content/drive/')
#file_path='./data/Complete_dataset/'
file_path='/content/drive/MyDrive/sepi_master_thesis/Complete_dataset/'


Mounted at /content/drive/


In [ ]:
# STEP 4: Copy your dataset from Drive to LLaMA-Factory/data folder
!mkdir -p data/qwen_bio_nli
!cp /content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train_format_a_statement_in_input.json data/qwen_bio_nli/train.json
!cp /content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev_format_a_statement_in_input.json data/qwen_bio_nli/dev.json


In [ ]:
# STEP 5: Create dataset_info.json so LLaMA-Factory knows how to read your dataset
import json, os
os.makedirs("data", exist_ok=True)
dataset_info = {
    "qwen_bio_nli": {
        "file_name": "qwen_bio_nli",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output"
        },
        "format": "alpaca"
    }
}
with open("data/dataset_info.json", "w") as f:
    json.dump(dataset_info, f, indent=2)


In [ ]:
# STEP 6: Create training config file (JSON)
train_config = {
    "stage": "sft",
    "do_train": True,
    "model_name_or_path": "Qwen/Qwen2-7B-Instruct",
    "dataset": "qwen_bio_nli",
    "template": "qwen",
    "finetuning_type": "lora",
    "lora_target": "all",
    "output_dir": "/content/drive/MyDrive/sepi_master_thesis/finetuned_models/qwen_bio_nli_lora",
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 10,
    "warmup_ratio": 0.1,
    "save_steps": 100,
    "learning_rate": 5e-5,
    "num_train_epochs": 3.0,
    "max_samples": 1700,
    "max_grad_norm": 1.0,
    "fp16": True,
    "use_liger_kernel": True
}
with open("train_qwen_bio_nli.json", "w") as f:
    json.dump(train_config, f, indent=2)


In [ ]:
# STEP 7: Start fine-tuning
!llamafactory-cli train train_qwen_bio_nli.json


2025-04-22 19:40:44.441132: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 19:40:44.458984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745350844.480198    2592 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745350844.486803    2592 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 19:40:44.508877: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# STEP 8: Merge LoRA adapters into the base model (recommended if you want a standalone model)
merge_config = {
    "model_name_or_path": "Qwen/Qwen2-7B-Instruct",
    "adapter_name_or_path": "/content/drive/MyDrive/sepi_master_thesis/finetuned_models/qwen_bio_nli_lora",
    "template": "qwen",
    "finetuning_type": "lora",
    "export_dir": "/content/drive/MyDrive/sepi_master_thesis/models/qwen2_7b_lora_merged",
    "export_size": 2,
    "export_device": "cpu"
}
with open("merge_qwen2_7b.json", "w") as f:
    json.dump(merge_config, f, indent=2)

!llamafactory-cli export merge_qwen2_7b.json

2025-04-22 20:35:06.948284: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 20:35:06.965418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745354106.986326   17080 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745354106.992658   17080 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 20:35:07.013672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# Run merge
!llamafactory-cli export merge_qwen2_7b.json

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = '/content/drive/MyDrive/sepi_master_thesis/models/qwen2_7b_lora_merged'

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, trust_remote_code=True) # Removed device_map="auto" as it's redundant when loading to CPU.
model.eval() # Set the model to evaluation mode to disable dropout and other training-specific features

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/sepi_master_thesis/models/qwen2_7b_lora_merged'. Use `repo_type` argument if needed.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
import json
from tqdm import tqdm
import pandas as pd

# Load test set
#test_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test_format_a_statement_in_input.json"
df = pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test-zero-shot.csv")

predictions = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Running model predictions"):
    full_prompt = row["text"]
    inputs = tokenizer(
    full_prompt,
    return_tensors="pt",
    truncation=True,
    max_length=4096
    ).to(model.device)
    outputs = model.generate(
        **inputs,
        eos_token_id=tokenizer.eos_token_id,  # <- important!
        pad_token_id=tokenizer.pad_token_id,
        max_new_tokens=3)
    # Decode and remove original input from the result
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    pred = decoded.replace(full_prompt, "").strip()

    predictions.append(pred)
df["predicted_output"] = predictions

Running model predictions:   0%|          | 0/10 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 24.12 MiB is free. Process 83010 has 14.71 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 132.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test-zero-shot.csv")
df

,text,label,predicted_output
0,Primary trial evidence are: ['Inclusion Criter...,Contradiction,contradiction
1,Primary trial evidence are: ['INTERVENTION 1: ...,Entailment,contradiction
2,Primary trial evidence are: ['Outcome Measurem...,Contradiction,entailment
3,Primary trial evidence are: ['Outcome Measurem...,Contradiction,contradiction
4,Primary trial evidence are: ['Outcome Measurem...,Entailment,contradiction
5,Primary trial evidence are: ['Adverse Events 1...,Contradiction,contradiction
6,Primary trial evidence are: ['Outcome Measurem...,Entailment,contradiction
7,Primary trial evidence are: ['DISEASE CHARACTE...,Contradiction,entailment
8,Primary trial evidence are: ['Outcome Measurem...,Entailment,contradiction
9,Primary trial evidence are: ['Inclusion Criter...,Contradiction,entailment
